In [1]:
import pandas as pd

In [2]:
catalogDf=pd.read_csv("summary_split.csv")
catalogDf.head()

,dna_type,filepath,ftp_basename,length_of_seqs,num_of_seqs
0,chromosome,/home/mitsuki/out/altorf/genome/fasta/GCF_0000...,GCF_000022385.1_ASM2238v1,2736272,1
1,plasmid,/home/mitsuki/out/altorf/genome/fasta/GCF_0000...,GCF_000022385.1_ASM2238v1,NaN,0
2,chromosome,/home/mitsuki/out/altorf/genome/fasta/GCF_0009...,GCF_000970085.1_ASM97008v1,5017558,1
3,plasmid,/home/mitsuki/out/altorf/genome/fasta/GCF_0009...,GCF_000970085.1_ASM97008v1,NaN,0
4,chromosome,/home/mitsuki/out/altorf/genome/fasta/GCF_0000...,GCF_000007345.1_ASM734v1,5751492,1


In [58]:
basedir="/home/mitsuki/out/altorf/genome/preprocess/comp/"
dct_lst=[]
for idx,record in catalogDf[catalogDf["num_of_seqs"]>0].iterrows():#exclude empty files
    dct={}
    dct["ftp_basename"]=record["ftp_basename"]
    dct["dna_type"]=record["dna_type"]
    
    filepath=basedir+record["ftp_basename"]+"_"+record["dna_type"]+".comp"
    with open(filepath,'r') as f:
        f.readline()#skip Total # n-mers header
        f.readline()#skip length

        for line in f:
            line=line.strip()
            if line in ["1-mer stats","2-mer stats","3-mer stats"]:
                continue
            else:
                line_splitted=line.split('\t')
                dct[line_splitted[0]]=line_splitted[1]
    dct_lst.append(dct)
out_df=pd.DataFrame(dct_lst)

(2122, 1006)


,A,AA,AAA,AAC,AAG,AAH,AAK,AAM,AAN,AAR,...,YY,YYC,YYK,YYR,YYS,YYT,YYW,YYY,dna_type,ftp_basename
0,888038,296646,95413,45414,64186,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chromosome,GCF_000022385.1_ASM2238v1
1,1420708,489353,184225,78562,108820,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chromosome,GCF_000970085.1_ASM97008v1
2,1638004,565926,214597,90759,124374,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chromosome,GCF_000007345.1_ASM734v1
3,306724,58443,9747,16996,21121,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chromosome,GCF_000350305.1_ASM35030v1
4,1340341,474673,180218,72157,99459,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chromosome,GCF_000969905.1_ASM96990v1


In [61]:
basicBase_tpl=('A','C','G','T')
columns_sorted=list(basicBase_tpl)

#add other single base
ambiguousBase_lst=[]
for column in df.columns:
    if(len(column)==1) and not(column in columns_sorted):
        ambiguousBase_lst.append(column)
ambiguousBase_lst.sort()
columns_sorted+=ambiguousBase_lst

#add 2-mer
for i in basicBase_tpl:
    for j in basicBase_tpl:
        columns_sorted.append(i+j)

for i in basicBase_tpl:
    for j in basicBase_tpl:
        for k in basicBase_tpl:
            columns_sorted.append(i+j+k)

In [78]:
out_df["num_basic"]=out_df[list(basicBase_tpl)].astype(float).sum(axis=1)
out_df["num_ambiguous"]=(out_df[ambiguousBase_lst].astype(float).sum(axis=1)).astype(int)
out_df["per_ambiguous"]=out_df["num_ambiguous"]*100/(out_df["num_basic"]+out_df["num_ambiguous"])
out_df["G+C"]=(out_df[['C','G']].astype(float).sum(axis=1))/out_df["num_basic"]

In [79]:
out_df=out_df[["ftp_basename","dna_type","G+C","num_ambiguous","per_ambiguous"]+columns_sorted]

In [80]:
out_df.head()

,ftp_basename,dna_type,G+C,num_ambiguous,per_ambiguous,A,C,G,T,B,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,GCF_000022385.1_ASM2238v1,chromosome,0.351110,0,0.0,888038,475131,485602,887501,NaN,...,14898,59802,45239,25365,35234,33745,95524,58476,47604,96167
1,GCF_000970085.1_ASM97008v1,chromosome,0.429404,0,0.0,1420708,1073050,1081510,1442290,NaN,...,52223,96142,93249,75156,63340,72350,91433,129173,89002,192140
2,GCF_000007345.1_ASM734v1,chromosome,0.426809,0,0.0,1638004,1228410,1226378,1658700,NaN,...,60400,112816,108481,86152,72519,82104,104484,149528,102469,220277
3,GCF_000350305.1_ASM35030v1,chromosome,0.583030,0,0.0,306724,427580,424288,302513,NaN,...,37083,22944,22468,21107,24589,17132,5507,26971,14986,9466
4,GCF_000969905.1_ASM96990v1,chromosome,0.397795,0,0.0,1340341,902862,889505,1373044,NaN,...,39862,94306,85991,64803,56747,65669,99640,117838,85862,191079


In [82]:
out_df.to_csv("summary_composition.csv",index=False)